### Decompress

In [1]:
import tarfile

fname = 'example/20211129_2021231.tar.gz' # Example on the latest (daily) package of XML files

if fname.endswith("tar.gz"):

    tar = tarfile.open(fname, "r:gz")
    folder_name  = tar.getnames()[0]
    file_names = tar.getnames()[1:]
    tar.extractall('./') # maybe we can use an extraction folder if we want to decompress and combine multipe files
    tar.close()

### Ectract and save to CSV file

In [21]:
from lxml import etree

NMSP = {"ted": "http://publications.europa.eu/resource/schema/ted/R2.0.9/publication"}

for xml in file_names:
      
    tree = etree.parse(xml)
    root = tree.getroot()

    doc_ID = []
    contractee = []

    if root.nsmap[None]  == 'http://publications.europa.eu/resource/schema/ted/R2.0.9/publication': # check from version R2.0.9 in Namespace
        
        if  root.xpath('ted:CODED_DATA_SECTION/ted:CODIF_DATA/ted:TD_DOCUMENT_TYPE/@CODE', namespaces=NMSP) == ['7']: # check for Contract award notice only (Code 7)

            
            for item in root.findall('ted:FORM_SECTION/ted:F03_2014[@CATEGORY = "ORIGINAL"]/ted:AWARD_CONTRACT', namespaces=NMSP): # loop over all awardes item / lots in the AWARD_CONTRACT section
                contractor = []
                value = []

                try:
                    value = item.findtext("ted:AWARDED_CONTRACT/ted:VALUES/ted:VAL_TOTAL", namespaces=NMSP) # extract value
                except: continue
                    
                try:
                    contractor = item.findtext('ted:AWARDED_CONTRACT/ted:CONTRACTORS/ted:CONTRACTOR/ted:ADDRESS_CONTRACTOR/ted:OFFICIALNAME', namespaces=NMSP) # extract contractor
                except: continue
                    
                try:
                    currency = item.xpath('ted:AWARDED_CONTRACT/ted:VALUES/ted:VAL_TOTAL/@CURRENCY', namespaces=NMSP) # extract currency
                except: continue

                try:
                    doc_ID = root.get('DOC_ID', default=None) # extract document ID
                    contractee = root.findtext('ted:FORM_SECTION/ted:F03_2014[@CATEGORY = "ORIGINAL"]/ted:CONTRACTING_BODY/ted:ADDRESS_CONTRACTING_BODY/ted:OFFICIALNAME', namespaces=NMSP) # extract contractee
                except: continue
                
                try:
                    with open('my_file.csv', 'a') as file:
                        file.write(f'{doc_ID},{contractee.replace(",", "")},{contractor.replace(",", "")},{float(value)},{currency}"\n"') # store everything in a CSV File
                except:
                    pass